In [1]:
import torch
import os
import numpy as np
import pandas as pd
import shutil

from tqdm import tqdm
from options import SamplingOptions
from torch.utils.data import DataLoader
from y_sample_dataset import SamplingDataset
from model import CHARACTERModel
from PIL import Image
from google.cloud import storage

In [2]:
def tensor2im(input_image, imtype=np.uint8):
    """"Converts a Tensor array into a numpy image array.

    Parameters:
        input_image (tensor) --  the input image tensor array
        imtype (type)        --  the desired type of the converted numpy array
    """
    if not isinstance(input_image, np.ndarray):
        if isinstance(input_image, torch.Tensor):  # get the data from a variable
            image_tensor = input_image.data
        else:
            return input_image
        image_numpy = image_tensor[0].cpu().float().numpy()  # convert it into a numpy array
        if image_numpy.shape[0] == 1:  # grayscale to RGB
            image_numpy = np.tile(image_numpy, (3, 1, 1))
        image_numpy = (np.transpose(image_numpy, (1, 2, 0)) + 1) / 2.0 * 255.0  # post-processing: tranpose and scaling
    else:  # if it is a numpy array, do nothing
        image_numpy = input_image
    return Image.fromarray(image_numpy.astype(imtype))

In [3]:
opt = SamplingOptions()
os.makedirs(opt.savefd, exist_ok=True)

In [4]:
dataset = SamplingDataset(opt)
loader = DataLoader(dataset, batch_size=opt.batch_size, shuffle=False, drop_last=False, num_workers=int(opt.num_threads))

In [5]:
model = CHARACTERModel(opt=opt)
model.load_model()

Param count for Ds initialized parameters: 19541696
Param count for Ds initialized parameters: 20591296
Param count for Ds initialized parameters: 27289027


In [6]:
# outer loop for different epochs; we save the model by <epoch_count>, <epoch_count>+<save_latest_freq>
model.eval() 
pbar = tqdm(loader)
for data in pbar:  # inner loop within one epoch         
    model.set_input(data)         # unpack data from dataset and apply preprocessing
    model.forward()
    generated = model.img_print2write
    for i in range(len(generated)):
        img = tensor2im(generated[[i]])
        font = model.writerID[i]
        letter = model.image_paths[i]
        path = f"{opt.savefd}/{opt.tag}__{font}__{letter}.png"
        img.save(path)
        pbar.set_postfix(path=path)

100%|██████████| 350/350 [01:23<00:00,  4.19it/s, path=/home/jupyter/ai_font/data/reports/exp0604/cggan/cggan_full__카엔플린__힣.png]
